In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

from keras.preprocessing.image import ImageDataGenerator

# 1. Data

In [2]:
# Preprocessing training set

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

train_set = train_datagen.flow_from_directory(
    'dataset/training_set/',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary'
)

Found 8000 images belonging to 2 classes.


In [3]:
# Preprocessing test set

test_datagen = ImageDataGenerator(
    rescale = 1./255
)

test_set = test_datagen.flow_from_directory(
    'dataset/test_set/',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.


# 2. Build CNN

In [4]:
cnn = tf.keras.models.Sequential()

# 1. Convolution
cnn.add(tf.keras.layers.Conv2D(
    filters = 32,
    kernel_size=3,
    activation='relu',
    input_shape=(64,64, 3) # The 3 is because of color (RGB); grayscale would be 1
))

# 2. Pooling
cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=(2,2), 
    strides=2
))

# 3. Second Convolution layer
cnn.add(tf.keras.layers.Conv2D(
    filters=32, 
    kernel_size=3, 
    activation='relu'
))

# 4. MaxPooling again
cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=2,
    strides=2
))

#5. Flattening - this will be input of future neural network
cnn.add(tf.keras.layers.Flatten())

#6. Fully connected layer
cnn.add(tf.keras.layers.Dense(
    units=128, 
    activation='relu', 
))

#7. Output layer
cnn.add(tf.keras.layers.Dense(
    units=1, 
    activation='sigmoid', 
))


# 3. Train CNN

In [28]:
# Compile
cnn.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Train and evaluate on test set
cnn.fit(
    x=train_set,
    validation_data = test_set,
    epochs = 25    
)

Epoch 1/25
250/250 [==============================] - 60s 239ms/step - loss: 0.4193 - accuracy: 0.8059 - val_loss: 0.4779 - val_accuracy: 0.7870
Epoch 2/25
250/250 [==============================] - 51s 203ms/step - loss: 0.3921 - accuracy: 0.8192 - val_loss: 0.4795 - val_accuracy: 0.7795
Epoch 3/25
250/250 [==============================] - 59s 234ms/step - loss: 0.3813 - accuracy: 0.8286 - val_loss: 0.4823 - val_accuracy: 0.7790
Epoch 4/25
250/250 [==============================] - 64s 258ms/step - loss: 0.3685 - accuracy: 0.8325 - val_loss: 0.4983 - val_accuracy: 0.7845
Epoch 5/25
250/250 [==============================] - 55s 220ms/step - loss: 0.3478 - accuracy: 0.8482 - val_loss: 0.5982 - val_accuracy: 0.7405
Epoch 6/25
250/250 [==============================] - 50s 201ms/step - loss: 0.3333 - accuracy: 0.8522 - val_loss: 0.5014 - val_accuracy: 0.7680
Epoch 7/25
250/250 [==============================] - 55s 220ms/step - loss: 0.3260 - accuracy: 0.8556 - val_loss: 0.4859 - val_ac

# 4. Test/Evaluate

In [25]:
from keras.preprocessing import image

# load image
test_image = image.load_img(
    'dataset/single_prediction/cat.4327.jpg',
    target_size=(64,64)
)

# Convert image from PIL to array
test_image = image.img_to_array(test_image)

# Image needs to be in the same format as training; training had a batch dimension of 32
# so need to add an extra dimension, which will convert image to batch
test_image = np.expand_dims(test_image, axis= 0) # 0 param indicates axis

# predict
result = cnn.predict(test_image/255)


print(train_set.class_indices)
num_prediction = result[0][0] # first 0  is to get inside the batch;
                          # the second 0 is the only element in batch

if num_prediction > 0.5:
    prediction = 'dog'
else:
    prediction ='cat'
    
print(result[0][0], prediction)

{'cats': 0, 'dogs': 1}
0.046929866 cat
